In [1]:
%matplotlib ipympl
import matplotlib as plt
import sys
sys.path.append('../../plotting')
from plotlib import * 
import numpy as np
import pandas as pd
import matplotlib
import glob
from math import sqrt
SPINE_COLOR = 'gray'
PATH = './../NDN/results/client2/2017112709:25:46_log'
import os
script = "./../NDN/Extract_KPIs.sh"
data = {}


FigureCanvasNbAgg()

In [2]:
def read_trace_file_url(trace_file_url):
    colnames = ['Timestamp','Increasingtime','GPS_1','GPS_2', 'bytes', 'elapsed_ms']
    data = pd.read_csv(trace_file_url, names=colnames, delim_whitespace=True)
    data['bitrate'] = data.bytes * 8 /  (data.elapsed_ms * 1000 )# mbit/s
    data['Increasingtime'] = (data['Increasingtime'] - data['Increasingtime'][0]) / 1000 - 288
    data.Timestamp = pd.to_datetime(data.Timestamp)
    return(data)


In [48]:

#Video Segment/Sampled throughput
def sampletroughput(PATH, kind='other'):
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        path = os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)),  "throughput")
        print(path)
        df = pd.read_csv(path, delim_whitespace=True ,names = ['seconds_offset', 'throughput'])
        df.throughput = df.throughput.apply(lambda x: x * 1.00e-6) # in Mbit/s
        title=f

        profile = read_trace_file_url('./report.2010-09-28_1407CEST.log.txt')
        sec=profile.Timestamp
        bw=profile.bitrate
        if kind == 'cdf':
            fig = df.plot("throughput", kind='hist',  bins=200, normed=1, histtype='step',
                           cumulative=True)
            fig.set_ylabel('Density')
            fig.set_xlabel('Mbit/s')
            #fig.set_xlim(0, 5)
        else:
            fig = df.plot(x="seconds_offset", y="throughput", kind='scatter', s=0.01)
            fig.set_xlabel('Playback time (seconds)')
            lines = fig.set_ylabel('Mbit/s')

        fig.plot(sec, bw, 'r--')

        fig.set_title(os.path.dirname(f))
        plt.tight_layout()
        plt.savefig("{0}_throughput{1}".format(f, ".pdf"))

#sampletroughput(PATH)

In [27]:
#NDN Chunk throughput

def ndnChunkTroghput(PATH, kind='dots'):
    N = 5
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        print(f)
        #global df
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "chunkThroughput") , delim_whitespace=True ,names = ['seconds_offset', 'chunkThroughput'])
        title=f

        #profile = pd.read_csv('./StandardDASH_Profile', delim_whitespace=True, names = ['seconds_offset', 'bandwidth', 'latency' ])
        #sec=profile.seconds_offset
        #bw=profile.bandwidth

        #x=df.seconds_offset
        #y=df.chunkThroughput
        #plt.hexbin(x, y, gridsize=(600,1000), yscale='log', mincnt = 1)
        #plt.tight_layout()
        #plt.savefig(f+"_kde_grid_logy.pdf")
        if kind == 'scatter':
            fig = df.plot(x="seconds_offset" , y="chunkThroughput", logy=False, kind='scatter', marker='x', s=0.05)
            fig.set_xlabel('Playback time (seconds)')
            fig.set_ylim(0,6)
            fig.set_xlim(0,200)
            fig.set_ylabel('Mbit/s')
        elif kind == 'hist':
            fig = df.plot(x="seconds_offset" , y="chunkThroughput", kind='hist', bins=50)
            fig.set_ylabel('???')
            fig.set_xlabel('Mbit/s')
        elif kind == 'kde':
            fig = df.plot("chunkThroughput", kind='kde')
            fig.set_ylabel('Density')
            fig.set_xlabel('Mbit/s')
            fig.set_xlim(0, 100)
        elif kind == 'cdf':
            fig = df.plot("seconds_offset", kind='hist',  bins=200, normed=1, histtype='step',
                           cumulative=True)
            fig.set_ylabel('Density')
            fig.set_xlabel('Mbit/s')
            fig.set_xlim(0, 10)
            
        #profile = read_trace_file_url('./ndn/report.2010-09-28_1407CEST.log.txt')
        #sec=profile['Increasingtime']
        #bw=profile.bitrate
        #profile
        #fig.plot(sec, bw, colors.mpl_colors[0])
        #fig.set_title(os.path.dirname(f))
        fig.legend({'NDN-Chunk based measurment', 'Bandwidth trace'})



        #data.update({os.path.dirname(f) : df})
        plt.tight_layout()
        #plt.savefig("{0}_scatter_natural_logy-{1}{2}".format(f,os.path.dirname(f).split('/')[2], ".pdf"))
        
        
#NDN Chunk throughput

def ndnChunkTroghputQuality(PATH):
    N = 5
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        print(f)
        #global df
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "chunkThroughput") , delim_whitespace=True ,names = ['seconds_offset', 'chunkThroughput'])
        title=f

        #profile = pd.read_csv('./StandardDASH_Profile', delim_whitespace=True, names = ['seconds_offset', 'bandwidth', 'latency' ])
        #sec=profile.seconds_offset
        #bw=profile.bandwidth

        #x=df.seconds_offset
        #y=df.chunkThroughput
        #plt.hexbin(x, y, gridsize=(600,1000), yscale='log', mincnt = 1)
        #plt.tight_layout()
        #plt.savefig(f+"_kde_grid_logy.pdf")
        fig = df.plot(x="seconds_offset" , y="chunkThroughput", logy=False, label='NDN-Chunk Troughput', kind='scatter', c='0.4', marker='x', s=0.08)
        fig.set_xlabel('Playback time (seconds)')
        fig.set_ylim(0,6)
        fig.set_xlim(0,200)
        fig.set_ylabel('Mbit/s')
            
        profile = read_trace_file_url('./ndn/report.2010-09-28_1407CEST.log.txt')
        sec=profile['Increasingtime']
        bw=profile.bitrate
        profile
        bw_plot = fig.plot(sec, bw, label='Bandwidth Trace')
        
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "quality") , delim_whitespace=True, 
                         names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])

        df.videobitrate = df.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
        df.realbitrate = df.realbitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s


        bitrate_plot = fig.plot(df.seconds_offset, df.videobitrate, 'r', label='Video Bitrate')
        
        #fig.set_title(os.path.dirname(f))
        #fig.legend([bw_plot, bitrate_plot], ['Video Bitrate',  'Bandwidth trace'])
        handles, labels = fig.get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
        fig.legend(handles, labels)
        fig.legend(handles, labels, ncol=3,  loc='upper center', frameon=True, mode="expand")
        #data.update({os.path.dirname(f) : df})
        plt.tight_layout()
        #plt.savefig("{0}_scatter_natural_logy-{1}{2}".format(f,os.path.dirname(f).split('/')[2].strip('.'), ".pdf"))

In [28]:

def qualityPlot(PATH):
    #Quality levels
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "quality") , delim_whitespace=True, 
                         names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])
        title=f
        df.videobitrate = df.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
        df.realbitrate = df.realbitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s

        #profile = pd.read_csv('./StandardDASH_Profile', delim_whitespace=True, names = ['seconds_offset', 'bandwidth', 'latency' ])
        #sec=profile.seconds_offset
        #bw=profile.bandwidth

        fig = df.plot(x="seconds_offset", y="realbitrate", )

       # fig.plot([1, 600], [1.03, 1.03], 'y:') 
       # fig.plot([1, 600], [1.24, 1.24], 'y:')
       # fig.plot([1, 600], [1.55, 1.55], 'y:')
       # fig.plot([1, 600], [2.13, 2.13], 'y:')
       # fig.plot([1, 600], [2.48, 2.48], 'y:')
       # fig.plot([1, 600], [3.08, 3.08], 'y:')
       # fig.plot([1, 600], [3.53, 3.53], 'y:')
       # fig.plot([1, 600], [3.84, 3.84], 'y:')
       # fig.plot([1, 600], [4.22, 4.22], 'y:')

        br = fig.plot(df.seconds_offset, df.videobitrate, 'r:', label='Selected Adaptation Bitrate')
        
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "bufferlevel") , delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
        df.bufferlevel = df.bufferlevel.apply(lambda x: x * 1.00e+2) # in Percentage
        #title=f
        bl = fig.plot(df.seconds_offset, df.bufferlevel, 'k--', label='Real Segment Bitrate')

        #fig.set_title(os.path.dirname(f))
        #fig.plot(sec, bw, 'k--')

        print(df.videobitrate.mean())

        fig.set_xlabel('Playback time (seconds)')
        #fig.set_title(os.path.dirname(f))
        handles, labels = fig.get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
        fig.legend(handles, labels, bbox_to_anchor=(0., 1.02, 1., 1.102), loc=1, ncol=1, mode="expand", borderaxespad=0.)
        #fig.legend([br, bl], ['Real Segment Bitrate', 'Selected Adaptation Bitrate'])
        #, 'Buffer fill level (percentage)'})

        lines = fig.set_ylabel('Video quality (Mbit/s)')
        #plt.tight_layout()
        #plt.savefig("{0}_quality-{1}{2}".format(f,os.path.dirname(f).split('/')[2].strip('.'), ".pdf"))

In [37]:
def bufferPlot(PATH):
    #Buffer fill level
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "bufferlevel") , delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
        df.bufferlevel = df.bufferlevel.apply(lambda x: x * 1.00e+2) # in Percentage
        title=f
        fig = df.plot(x="seconds_offset", y="bufferlevel", )
        fig.set_xlabel('Playback time (seconds)')
        lines = fig.set_ylabel('Buffer fill level (percentage)')
        plt.tight_layout()
        fig.legend({})
        #fig.set_title(os.path.dirname(f))

        #plt.savefig("{0}_bufferlevel-{1}{2}".format(f,os.path.dirname(f).split('/')[2].strip('.'), ".pdf"))

In [42]:

        #ax.set_ylabel('Mbit/s')
        

In [ ]:
            
        profile = read_trace_file_url('./ndn/report.2010-09-28_1407CEST.log.txt')
        sec=profile['Increasingtime']
        bw=profile.bitrate
        profile
        bw_plot = ax.plot(sec, bw, label='Bandwidth Trace')
        
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "quality") , delim_whitespace=True, 
                         names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])

        df.videobitrate = df.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
        df.realbitrate = df.realbitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s


        bitrate_plot = ax.plot(df.seconds_offset, df.videobitrate, 'r', label='Video Bitrate')
        
        #fig.set_title(os.path.dirname(f))
        #fig.legend([bw_plot, bitrate_plot], ['Video Bitrate',  'Bandwidth trace'])
        handles, labels = ax.get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
        fig.legend(handles, labels)
        fig.legend(handles, labels, ncol=3,  loc='upper center', frameon=True, mode="expand")
        #data.update({os.path.dirname(f) : df})
        plt.tight_layout()
        #plt.savefig("{0}_scatter_natural_logy-{1}{2}".format(f,os.path.dirname(f).split('/')[2].strip('.'), ".pdf"))

In [71]:
import re
def ndnChunkTroghputQuality2(PATH, fig, ax):
    N = 5
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        #fig, ax = plt.subplots(1,1)
        format_axes(ax)
        #global df
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "chunkThroughput") , delim_whitespace=True ,names = ['seconds_offset', 'chunkThroughput'])
        df.columns = map(lambda x: re.sub('_', '', x), df.columns)
        #df['subject'] = 0
        df = df.sample(frac=0.5)
        title=f
        #display(df.head())
        ax.scatter(df["secondsoffset"], df["chunkThroughput"], marker='x', s=0.1, color=Accent_5.mpl_colors[1])
        #sns.tsplot(data=df, unit='subject', interpolate=True, ci=[68, 95], time="secondsoffset", value="chunkThroughput")
        ax.set_xlabel('Playback time (seconds)')
        ax.set_ylabel('Video quality (Mbit/s)')
        ax.set_ylim(0,6)
        ax.set_xlim(0,200)
        profile = read_trace_file_url('./ndn/report.2010-09-28_1407CEST.log.txt')
        sec=profile['Increasingtime']
        bw=profile.bitrate
        profile
        bw_plot = ax.plot(sec, bw, label='Bandwidth Trace')
        
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "quality") , delim_whitespace=True, 
                         names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])

        df.videobitrate = df.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
        df.realbitrate = df.realbitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
        bitrate_plot = ax.plot(df.seconds_offset, df.videobitrate, 'r', label='Video Bitrate')
        fig.tight_layout()
        
compPaths = [
            ('BOLA','2017.12.22-14:41-bolasequential_log'),
             ('Panda', '2017.12.21-11:15-pandasequential_log'),
#             ('Extended Panda', '2017.12.22-14:08-psequential_log'),
             ('Extended Panda', '2017.12.21-09:04-psequential_log')]

size = setupfiguresthin()
fig, axes = plt.subplots(3,2)
fig.set_size_inches(size[0], size[1]*3)
j = 0
for player in compPaths:
    for i in range(1,3):
        list_of_files = glob.glob('./results/client{0}/{1}'.format(i, player[1])) # * means all if need specific format then *.csv
        latest_file = sorted(list_of_files, key=os.path.getctime)[0]
        setupfigures80()
        axes[j][i-1].set_title('{}, Player {}'.format(player[0], i))
        ndnChunkTroghputQuality2(latest_file, fig, axes[j][i-1])
        #qualityPlot(PATH)
        #bufferPlot(PATH)
    j = j + 1

axes[0][0].legend(bbox_to_anchor=(0., 1.2, 2.2, .102), loc=3,
               ncol=4, mode="expand", borderaxespad=0)
fig.subplots_adjust(top=0.8)
fig.savefig(getFilename('ndn-scatter-segments-combined', 'ndn'))

/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

./results/client1


/opt/conda/lib/python3.6/site-packages/pandas/io/parsers.py:677: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


./results/client2
./results/client1
./results/client2
./results/client1
./results/client2
./gfx/ndn/ndn-scatter-segments-combined.pdf


#gOut/gIn and Chunk Throughput
for f in glob.iglob('./*10-*/**/chunkThroughputRttQuotient', recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
    df = pd.read_csv(f, delim_whitespace=True, names = ['seconds_offset', 'rtt', 'quotient', 'chunkThroughput'])

    
    #df['movingavgTP'] = pd.rolling_mean(df.chunkThroughput, 100)
    #df['movingavgQ'] = pd.rolling_mean(df.quotient, 100)
    #fig = df.plot.scatter(x="movingavgTP", y="movingavgQ", s=0.001)
    fig = df.plot.scatter(x="chunkThroughput", y="quotient", s=0.001)
    
    #fig.set_xscale('log')
    #fig.set_yscale('log')
    fig.set_xbound(0,10)
    fig.set_ybound(0,3)
    
    fig.set_xlabel('Throughput (Mbit/s)')
    fig.set_ylabel('gOut/gIn')
    plt.tight_layout()
    f = f[:-26]
    plt.savefig(f+"gOutgIn2chunkThroughput_natural_limxlimy3.pdf")

#RTT and Chunk Throughput
for f in glob.iglob('./*10-*/**/chunkThroughputRttQuotient', recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
    df = pd.read_csv(f, delim_whitespace=True, names = ['seconds_offset', 'rtt', 'quotient', 'chunkThroughput'])
    df.rtt = df.rtt.apply(lambda x: x * 1.00e-3) # in ms
    
    #df['movingavgTP'] = pd.rolling_mean(df.chunkThroughput, 100)
    #df['movingavgRTT'] = pd.rolling_mean(df.rtt, 100)
    #fig = df.plot.scatter(x="movingavgTP", y="movingavgRTT", s=0.001)
    fig = df.plot.scatter(x="chunkThroughput", y="rtt", s=0.001)
    
    #fig.set_xscale('log')
    #fig.set_yscale('log')
    fig.set_xbound(0,10)
    
    fig.set_xlabel('Throughput (Mbit/s)')
    fig.set_ylabel('RTT (ms)')
    plt.tight_layout()
    f = f[:-26]
    plt.savefig(f+"rtt2chunkThroughput_natural_limx.pdf")

#BW with RTT + throughput with DataDelay in one Plot
for f in glob.iglob('./*10-*/**/classicBW2throughputEstimate', recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
    df = pd.read_csv(f, delim_whitespace=True, names = ['seconds_offset', 'classicBW', 'chunkThroughput'])
    title=f
    
    df['movingavgBW'] = pd.rolling_mean(df.classicBW, 100)
    df['movingavgTP'] = pd.rolling_mean(df.chunkThroughput, 100)
    ax1 = df.plot(kind='line', x='seconds_offset', y='movingavgBW', color='r')
    fig = df.plot(kind='line', x='seconds_offset', y='movingavgTP', color='b', ax=ax1)
    
    
    #ax1 = df.plot(kind='line', x='seconds_offset', y='classicBW', color='r')
    #fig = df.plot(kind='line', x='seconds_offset', y='chunkThroughput', color='b', ax=ax1)

    fig.set_xlabel('Playback time (seconds)')
    fig.set_ylabel('Mbit/s')

    plt.tight_layout()
    f = f[:-18]
    plt.savefig(f+"tpEstimate_mvgavg.pdf")

#Sampled buffer level comparison of different samplesizes (INIT CHUNK PROBLEM)
db5 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_5_p_44_171004-1535_out/bufferlevel', delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
db30 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_30_p_44_171004-1546_out/bufferlevel', delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
db50 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_50_p_44_171004-1556_out/bufferlevel', delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
db100 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_100_p_44_171004-1607_out/bufferlevel', delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])
dbS = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_p_44_171004-1520_out/bufferlevel', delim_whitespace=True, names = ['seconds_offset', 'bufferlevel'])

x=db5.seconds_offset
x2=db100.seconds_offset
y1=db5.bufferlevel
y2=db30.bufferlevel
y3=db50.bufferlevel
y4=db100.bufferlevel
y5=dbS.bufferlevel

plt.plot(x,y1, 'y:')
#plt.plot(x,y2, 'g:')
plt.plot(x,y3,'b:')
#plt.plot(x2,y4,'k:')
plt.plot(x, y5, 'r--')

axes = plt.gca()
axes.set_ylim([-0.05,1])
axes.set_xlim([-5,605])

plt.legend(['5 Samples', '50 Samples', 'Segment'])
plt.xlabel('Playback time (seconds)')
plt.ylabel('Buffer fill level')
    
plt.tight_layout()
plt.savefig("./presentation_figures/presentation_prob-bufferlevel.pdf")

#Sampled quality level comparison of different samplesizes (INIT CHUNK PROBLEM)
dq5 = pd.read_csv(os.path.join(os.path.dirname(PATH), "{0}_out".format(os.path.basename(PATH)), 'quality'), delim_whitespace=True, 
                     names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])
dq30 = pd.read_csv(os.path.join(os.path.dirname(PATH).replace("1", "2"), "{0}_out".format(os.path.basename(PATH)), "quality"), delim_whitespace=True, 
                     names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate', 'useless', 'realbitrate'])
#dq50 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_50_p_44_171004-1556_out/quality', delim_whitespace=True, names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate'])
#dq100 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_100_p_44_171004-1607_out/quality', delim_whitespace=True, names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate'])
#dqS = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_p_44_171004-1520_out/quality', delim_whitespace=True, names = ['seconds_offset', 'useless', 'useless', 'segment_number', 'qualitylevel', 'videobitrate'])

dq5.videobitrate = dq5.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
dq30.videobitrate = dq30.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
#dq50.videobitrate = dq50.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
#dq100.videobitrate = dq100.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s
#dqS.videobitrate = dqS.videobitrate.apply(lambda x: x * 1.00e-6) # in Mbit/s

x1=dq5.seconds_offset
x2=dq30.seconds_offset
y1=dq5.videobitrate
y2=dq30.videobitrate
#y3=dq50.videobitrate
#y4=dq100.videobitrate
#y5=dqS.videobitrate

plt.plot(x1,y1, 'y:')
#plt.plot(x,y2, 'g:')
plt.plot(x2,y2, 'b:')
#plt.plot(x,y4, 'k:')
#plt.plot(x,y5, 'r--')

#profile = pd.read_csv('./StandardDASH_Profile', delim_whitespace=True, names = ['seconds_offset', 'bandwidth', 'latency' ])
#sec=profile.seconds_offset
#bw=profile.bandwidth
#plt.plot(sec, bw, '#888888')

axes = plt.gca()
#axes.set_xlim([-5,600])
    
#plt.legend(['5 Samples', '30 Samples', '50 Samples', '100 Samples', 'Segment', 'BW Profile'])
#plt.legend(['5 Samples', '50 Samples', 'Segment', 'BW Profile'])
plt.xlabel('Playback time (seconds)')
plt.ylabel('Quality level (Mbit/s)')
    
plt.tight_layout()
plt.savefig("./results/{0}presentation_prob-quality.pdf".format(os.path.basename(PATH)))

#for the table
print(dq5.videobitrate.mean(), "\n",
      dq30.videobitrate.mean(), "\n",
      dq50.videobitrate.mean(), "\n",
      dq100.videobitrate.mean(), "\n",
      dqS.videobitrate.mean(),"\n")

#Sampled throughput comparison of different samplesizes (INIT CHUNK PROBLEM)
dt5 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_5_p_44_171004-1535_out/throughput', delim_whitespace=True, names = ['seconds_offset', 'throughput'])
dt30 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_30_p_44_171004-1546_out/throughput', delim_whitespace=True, names = ['seconds_offset', 'throughput'])
dt50 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_50_p_44_171004-1556_out/throughput', delim_whitespace=True, names = ['seconds_offset', 'throughput'])
dt100 = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_c_100_p_44_171004-1607_out/throughput', delim_whitespace=True, names = ['seconds_offset', 'throughput'])
dtS = pd.read_csv('./results_10-04_WITH-initChunk-used/logNDN_p_44_171004-1520_out/throughput', delim_whitespace=True, names = ['seconds_offset', 'throughput'])


dt5.throughput = dt5.throughput.apply(lambda x: x* 1.00e-6) # in Mbit/s
dt30.throughput = dt30.throughput.apply(lambda x: x * 1.00e-6) # in Mbit/s
dt50.throughput = dt50.throughput.apply(lambda x: x * 1.00e-6) # in Mbit/s
dt100.throughput = dt100.throughput.apply(lambda x: x * 1.00e-6) # in Mbit/s
dtS.throughput = dtS.throughput.apply(lambda x: x * 1.00e-6) # in Mbit/s

x1=dt5.seconds_offset
y1=dt5.throughput
x2=dt30.seconds_offset
y2=dt30.throughput
x3=dt50.seconds_offset
y3=dt50.throughput
x4=dt100.seconds_offset
y4=dt100.throughput
x5=dtS.seconds_offset
y5=dtS.throughput

profile = pd.read_csv('./StandardDASH_Profile', delim_whitespace=True, names = ['seconds_offset', 'bandwidth', 'latency' ])
sec=profile.seconds_offset
bw=profile.bandwidth
plt.plot(sec, bw, '#888888',  linewidth=1.0, zorder=2)

plt.scatter(x1,y1,color='y', marker='x', zorder=1)
#plt.scatter(x2,y2,color='g', marker='o')
plt.scatter(x3,y3,color='b', marker='x', zorder=4)
#plt.scatter(x4,y4,color='k', marker='o')
plt.scatter(x5,y5,color='r', marker='x', zorder=3)

axes = plt.gca()
#axes.set_ylim([1,7])
axes.set_xlim([-25,600])

plt.legend(['BW Profile', '5 Samples', '50 Samples','Segment'])
plt.xlabel('Playback time (seconds)')
plt.ylabel('Estimated Throughput (Mbit/s)')
    
plt.tight_layout()
plt.savefig("./presentation_figures/presentation_prob-throughput.pdf")

In [107]:
profile = read_trace_file_url('./../NDN/report.2010-09-28_1407CEST.log.txt')
def ndnChunkData(PATH, data):
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "chunkThroughput") , delim_whitespace=True ,names = ['seconds_offset', 'chunkThroughput'])
        data.update({os.path.dirname(f) : df})
        
import seaborn as sns
import matplotlib.ticker as ticker
#sns.set_style("ticks")
#sns.set_context("paper")
#sns.despine(left=True, bottom=False)

out = []
labels = []
lines = []
#compPaths = [('BOLA','2017.12.22-14:41-bolasequential_log'),
#             ('Panda', '2017.12.21-11:15-pandasequential_log'),
#             ('Extended Panda', '2017.12.21-09:04-psequential_log')]
res1 = []
res2 = []
res = []
compPaths = [('BOLA','2017.12.22-14:41-bolasequential_log'),
             ('Panda', '2017.12.21-11:15-pandasequential_log'),
             ('Extended Panda', '2017.12.22-14:08-psequential_log')]
for alg, file in compPaths:
    data = {}
    for i in range(1,3):
        ndnChunkData('./../NDN/results/client{0}/{1}'.format(i, file), data)
    res1.append(pd.Series(data['./../NDN/results/client1'].chunkThroughput, name='{} First Client'.format(alg)))
    res2.append(pd.Series(data['./../NDN/results/client2'].chunkThroughput, name='{} Second Client'.format(alg)))
#hist_kws=dict(cumulative=True, range=(0,20), histtype='step'),


./../NDN/results/client1
./../NDN/results/client2
./../NDN/results/client1
./../NDN/results/client2
./../NDN/results/client1
./../NDN/results/client2


In [108]:
#NDN Chunk throughput
from statsmodels.distributions.empirical_distribution import ECDF
#plt.set_cmap(palettable.tableau.TableauLight_10.get_mpl_colormap())
fig, ax = plt.subplots(2,1, sharex=True)
size = setupfiguresthin()
fig.set_size_inches(size[0], size[1]*2)
#latexify(fig_width=3.5, fig_height=3.5)
func_amr = lambda t: 1.0 - t
i = 0
for data in res1:
    ecdf = ECDF(data)
    ax[0].plot(ecdf.x, ecdf.y, color=COLOR_DEFAULT_TEHSIS.mpl_colors[i], label=data.name)
    i = i +1
    #data.plot(kind='hist', range=(0,50), bins=1000, normed=1, histtype='step', cumulative=True, ax=ax[0], legend=True, label=data.name,)
    #sns.distplot(a=data, axlabel='', norm_hist=True, bins=200, hist=True,   label=data.name, kde=False, rug=True, ax=ax[0])

i = 0              
for data in res2:
    ecdf = ECDF(data)
    ax[1].plot(ecdf.x, ecdf.y, label=data.name, linestyle='--', color=COLOR_DEFAULT_TEHSIS.mpl_colors[i])
    i = i +1
    #data.plot(kind='hist', range=(0,50), bins=1000, normed=1, histtype='step', cumulative=True, ax=ax[1], legend=True, label=data.name)
    #sns.distplot(a=data, axlabel='', norm_hist=True, bins=200, hist=True, label=data.name, kde=False, rug=False, ax=ax[1])

ax1 = fig.get_axes()[0]
ax2 = fig.get_axes()[1]
format_axes(ax1)
format_axes(ax2)
ecdf = ECDF(profile.bitrate)
ax1.plot(ecdf.x, ecdf.y, label='Trace', linestyle='dotted', color=COLOR_DEFAULT_TEHSIS.mpl_colors[i])
ax2.plot(ecdf.x, ecdf.y,linestyle='dotted', color=COLOR_DEFAULT_TEHSIS.mpl_colors[i])
ax2.set_xlabel('Mbit/s')
for axel in fig.get_axes():
    axel.set_xscale('log')
    axel.set_yscale('log')
    axel.set_ylim(0.05, 1)
    axel.set_xlim(0.1, 10)
    #axel.set_ylabel('CDF',labelpad=-4)

    axel.set_ylabel('ECDF')
fig.tight_layout() 
fig.subplots_adjust(top=0.85)
fig.legend(ncol=2, loc='upper center')
fig
plt.savefig(getFilename(name='ndn-cdf-pandabola', FOLDER='ndn'))

/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

./gfx/ndn/ndn-cdf-pandabola.pdf


In [ ]:
df = pd.read_csv("/Users/dstohr/Projects/containernet/results/client2/2017.12.12-15:43-psequential_cwindow", sep="\t", names=['time', 'val'])

In [ ]:
def plotCW(PATH):
    df = pd.read_csv(PATH, names=['time', 'val'])
    #from scipy import stats
    #df = df[(np.abs(stats.zscore(df)) < 2).all(axis=1)]
    df.plot(x='time', y='val')

In [ ]:
for i in range(1,3):
    list_of_files = glob.glob('./results/client{0}/*_cwindow'.format(i)) # * means all if need specific format then *.csv
    #print (list_of_files)
    latest_file = max(list_of_files, key=os.path.getctime)
    PATH = latest_file
    print(PATH)

    plotCW(PATH)

In [ ]:
for el in data:
    print(el)
    display(data[el].describe())
    

In [ ]:
sizes = pd.read_csv('segmentsizes.csv')

In [ ]:
import seaborn as sns
sns.set_style("ticks")
sns.set_context("paper")

segs = pd.read_csv('./segmentsizes.csv', names = ['offset', 'qualitylevel', 'id', 'scale', 'size'])

segs.size = segs['size'].apply(lambda x: x *0.0005) # from Kbits/2s to bps

fig = segs.boxplot(column='size', by='qualitylevel', showfliers=False, grid=False,)
#fig = sns.boxplot(x="size", y="qualitylevel", data=segs, linewidth=2.5, ax=fig)
format_axes(fig)
#plt.gca().yaxis.grid(True)

plt.title('')
plt.suptitle('')
sns.despine(offset=10, trim=True);
plt.xlabel('Quality Layer')
plt.ylabel('Segment Video\nBitrate (Mbit/s)')

plt.tight_layout()
plt
plt.savefig("./seg_variance_boxplot.pdf")

segs.groupby('qualitylevel').var()

In [95]:
profile = read_trace_file_url('./../NDN/report.2010-09-28_1407CEST.log.txt')
def ndnChunkData(PATH, data):
    for f in glob.iglob(PATH, recursive=True): # /*directoryRegExp*/ to reduce output (long runtime)
        ! {script} {os.path.dirname(f)} {os.path.basename(f)}
        df = pd.read_csv(os.path.join(os.path.dirname(f), "{0}_out".format(os.path.basename(f)), "chunkThroughput") , delim_whitespace=True ,names = ['seconds_offset', 'chunkThroughput'])
        data.update({os.path.dirname(f) : df})
        
import seaborn as sns
import matplotlib.ticker as ticker
#sns.set_style("ticks")
#sns.set_context("paper")
#sns.despine(left=True, bottom=False)

out = []
labels = []
lines = []
#compPaths = [('BOLA','2017.12.22-14:41-bolasequential_log'),
#             ('Panda', '2017.12.21-11:15-pandasequential_log'),
#             ('Extended Panda', '2017.12.21-09:04-psequential_log')]
res1 = []
res2 = []
res = []
compPaths = [
                ('BOLA','2017.12.14-13:36-psequential_log'),

            ('BOLA','2017.12.14-13:03-psequential_log'),
             ('Panda', '2017.12.14-09:13-bolasequential_log'),
        #     ('Extended Panda', '2017.12.13-13:10-psequential_log')
            ]
for alg, file in compPaths:
    data = {}
    for i in range(1,3):
        ndnChunkData('./../NDN/results/client{0}/{1}'.format(i, file), data)
    #res1.append(pd.Series(data['./../NDN/results/client1'].chunkThroughput, name='{} First Client'.format(alg)))
    res1.append(pd.Series(data['./../NDN/results/client2'].chunkThroughput, name='{} Second Client'.format(alg)))
#hist_kws=dict(cumulative=True, range=(0,20), histtype='step'),


./../NDN/results/client2
./../NDN/results/client2
./../NDN/results/client1
./../NDN/results/client2


In [96]:
#NDN Chunk throughput
from statsmodels.distributions.empirical_distribution import ECDF
#plt.set_cmap(palettable.tableau.TableauLight_10.get_mpl_colormap())
fig, ax = plt.subplots(1,1, sharex=True)
size = setupfiguresthin()
i = 0
for data in res1:
    ecdf = ECDF(data)
    ax.plot(ecdf.x, ecdf.y, color=COLOR_DEFAULT_TEHSIS.mpl_colors[i], label=data.name)
    i = i +1
    #data.plot(kind='hist', range=(0,50), bins=1000, normed=1, histtype='step', cumulative=True, ax=ax[0], legend=True, label=data.name,)
    #sns.distplot(a=data, axlabel='', norm_hist=True, bins=200, hist=True,   label=data.name, kde=False, rug=True, ax=ax[0])

ax1 = fig.get_axes()[0]
format_axes(ax1)
ecdf = ECDF(profile.bitrate)
#ax1.plot(ecdf.x, ecdf.y, label='Trace', linestyle='dotted', color=COLOR_DEFAULT_TEHSIS.mpl_colors[i])
ax1.set_xlim(0,50)
fig.tight_layout() 
fig.subplots_adjust(top=0.85)
fig.legend(ncol=2, loc='upper center')
#plt.savefig(getFilename(name='ndn-cdf-pandabola', FOLDER='ndn'))

/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()